In [203]:
import vtk

In [204]:
reader = vtk.vtkDICOMImageReader() 
reader.SetDirectoryName('Marching_Man')
reader.Update()

In [205]:
xMin, xMax, yMin, yMax, zMin, zMax = reader.GetDataExtent()
xSpacing, ySpacing, zSpacing = reader.GetOutput().GetSpacing()
print(xSpacing, ySpacing, zSpacing)
x0, y0, z0 = reader.GetOutput().GetOrigin()
center = [x0 + xSpacing * 0.5 * (xMin + xMax),
          y0 + ySpacing * 0.5 * (yMin + yMax),
          z0 + zSpacing * 0.5 * (zMin + zMax)]

0.9375 0.9375 1.5


In [206]:
axial = vtk.vtkMatrix4x4()
axial.DeepCopy((1, 0, 0, center[0],
                0, 1, 0, center[1],
                0, 0, 1, center[2],
                0, 0, 0, 1))

In [207]:
reslice = vtk.vtkImageReslice()
reslice.SetInputConnection(reader.GetOutputPort())
reslice.SetOutputDimensionality(2)
reslice.SetResliceAxes(axial)
reslice.SetInterpolationModeToCubic()

In [208]:
glob_level = 0.
glob_window = 2048.
table = vtk.vtkLookupTable()
table.SetRange(glob_level-glob_window/2, glob_level+glob_window/2) # Level-Window -> intensity range
table.SetValueRange(0.0, 1.0) # von schwarz nach weiß
table.SetSaturationRange(0.0, 0.0) # keine Farbsättigung
table.SetRampToLinear() # Linearer Verlauf
table.Build()

In [209]:
color = vtk.vtkImageMapToColors()
color.SetLookupTable(table)
color.SetInputConnection(reader.GetOutputPort())

In [210]:
image  = color.GetOutput()
mc = vtk.vtkMarchingCubes()
mc.ComputeNormalsOn()
mc.ComputeGradientsOn()
mc.SetValue(500, 1000) # hier noch richtig stellen
mc.SetInputData(image)
mc.Update()

In [211]:
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(mc.GetOutputPort())
mapper.ScalarVisibilityOff()

actor_property = vtk.vtkProperty()
actor_property.SetColor(1,1,1)
actor_property.SetOpacity(0.7)


actor = vtk.vtkActor()
actor.SetProperty(actor_property)
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
#renderer.SetBackground(1,1,1)
window = vtk.vtkRenderWindow()
window.AddRenderer(renderer)

In [215]:
interactorStyle = vtk.vtkInteractorStyleImage()
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetInteractorStyle(interactorStyle)
window.SetInteractor(interactor)
window.Render()

In [213]:
actions = {}
actions["Slicing"] = 0

def ButtonCallback(obj, event):
    if event == "LeftButtonPressEvent":
        actions["Slicing"] = 1
    else:
        actions["Slicing"] = 0

def MouseMoveCallback(obj, event):

    (lastX, lastY) = interactor.GetLastEventPosition()
    (mouseX, mouseY) = interactor.GetEventPosition()
    if actions["Slicing"] == 1:
        deltaY = mouseY - lastY
        reslice.Update()
        sliceSpacing = reslice.GetOutput().GetSpacing()[2]
        matrix = reslice.GetResliceAxes()
# move the center point that we are slicing through
        center = matrix.MultiplyPoint((0, 0, sliceSpacing*deltaY, 1))
        matrix.SetElement(0, 3, center[0])
        matrix.SetElement(1, 3, center[1])
        matrix.SetElement(2, 3, center[2])
        window.Render()
    else:
        interactorStyle.OnMouseMove()

interactorStyle.AddObserver("MouseMoveEvent", MouseMoveCallback)
interactorStyle.AddObserver("LeftButtonPressEvent", ButtonCallback)
interactorStyle.AddObserver("LeftButtonReleaseEvent", ButtonCallback)

6

In [214]:
interactor.Start()